### Latex macros:
$$
\newcommand{\E}{\text{E}}
\newcommand{\mbf}{\mathbf}
\newcommand{\bo}{\mathbf}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Var}{\text{Var}}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\e}{\frac{1}{\sigma^2_e}}
\newcommand{\f}{\frac{1}{\sigma^2_{\alpha}}}
$$

In [1]:
macro javascript_str(s) display("text/javascript", s); end
javascript"""
    MathJax.Hub.Config({
      TeX: { equationNumbers: { autoNumber: "AMS" } }
    });
    MathJax.Hub.Queue( 
        ["resetEquationNumbers",MathJax.InputJax.TeX], 
        ["PreProcess",MathJax.Hub], 
        ["Reprocess",MathJax.Hub] 
    );
"""

# Iterative methods for solving linear systems

Consider the system of consistent linear equations: 

$$
\mathbf{Ax} = \mathbf{b},
$$
Three iterative methods that we will use for solving the linear systems are the Jacobi method, the Gauss-Seidel Methods, and the Preconditioned Conjugate Gradient (PCCG) method. These methods can be used to solve normal equations shown in the previous section and Mixed Model Equations (MME) we will covered later. Consider MME, the left-hand-side (LHS) of the MME is represented by $\mathbf{A}$ and the right-hand-side (RHS) by $\mathbf{b}$. The LHS of the MME is often too large to store in memory as a “fully-stored” matrix. However, $\mathbf{A}$ is often very sparse. Thus, it is may be possible to store only the non-zero elements of $\mathbf{A}$ and compute $\mathbf{Ax_n}$, using sparse matrix methods.

### The conjugate gradient method

In the conjugate gradient method, the solution at iteration $n+1$ is: 

\begin{equation}
  \mathbf{x}_{n+1} = \mathbf{x}_n + \alpha_n\mathbf{d}_n, 
\end{equation}
where
\begin{equation}
\label{eq:5}
\alpha_n = \frac{-\mathbf{r}_n'\mathbf{r}_n}{\mathbf{d}_n'\mathbf{Ad}_n},
\end{equation}

\begin{equation}
  \label{eq:6}
  \mathbf{r}_n = \mathbf{Ax}_n - \mathbf{b},
\end{equation}

\begin{equation}
  \label{eq:7}
  \mathbf{d}_n = \mathbf{r}_n - \beta_{n-1}\mathbf{d}_{n-1}, 
\end{equation}

and

\begin{equation}
  \label{eq:8}
  \beta_{n-1} = \frac{-\mathbf{r}_n'\mathbf{r}_n}
                {\mathbf{r}'_{n-1}\mathbf{r}_{n-1}}.
\end{equation}

It can be shown that the residual can be computed as
\begin{equation}
  \label{eq:9}
  \mathbf{r}_n = \mathbf{r}_{n-1} + \alpha_{n-1}\mathbf{Ad}_{n-1},
\end{equation}

and thus avoiding computation of $\mathbf{Ax}_n$. However, using (\ref{eq:9})
leads to the accumulation of rounding errors. Thus, it is recommended that (\ref{eq:6}) is used every 50 iterations.

Unlike the Jacobi method, it is not intuitively obvious why the conjugate 
gradient method works. Following is an attempt to explain why the method works.

In the conjugate gradient method, the value of $\bo{x}$ that minimizes
\begin{equation}
\label{eq:10}
\begin{split}
f(\bo{x}) &= \frac{1}{2}\bo{x}'\bo{Ax} -\bo{b}'\bo{x}\\
\end{split}
\end{equation}
is obtained by line minimizations in $n$ linearly independent directions, where
$n$ is the order of the symmetric positive definite matrix $\bo{A}$. Note that
after minimization of $f(\bo{x})$ in any direction $\bo{d}_i$, the gradient
$\bo{r}_{i+1}$ will be orthogonal to $\bo{d}_i$. 

In the conjugate gradient method, each direction $\bo{d}_i$ is is chosen such
that the gradient $\bo{r}_{i+1}$ will also be orthogonal to all the directions
$\bo{d}_j$, for $j<i$, that have already been used for line minimization. If the
directions are also linearly independent, after $n$ line minimizations, the
function will be at its minimum in $n$ linearly independent directions. Further,
at this point, the gradient 
$$
\bo{r} = \bo{Ax} - \bo{b}
$$
is orthogonal to the $n$ direction vectors. Thus, it must be the zero vector.

Following is a description of how the direction vectors are computed.  Suppose
the search is initiated at $\bo{x}_0 = \bo{0}$. At this point, the gradient of
the $f(\bo{x})$ is
\begin{equation}
\label{eq:11}
  \begin{split}
    \bo{r}_0 &= \bo{Ax}_o - \bo{b}\\
            &= -b. 
  \end{split}
\end{equation}
Let $\bo{d}_0 = \bo{r}_0$ be the first direction for line minimization. After
minimization of $f(\bo{x})$ in the direction $\bo{d}_0$, the value of $\bo{x}$
is
\begin{equation}
  \label{eq:12}
  \bo{x}_1 = \bo{x}_0 + \alpha_0\bo{d}_0.
\end{equation}
At $\bo{x}_1$, the gradient of the function is
\begin{equation}
  \label{eq:13}
  \begin{split}
    \bo{r}_1 &= \bo{Ax}_1 - \bo{b}\\
            &= \bo{A}(\bo{x}_0 + \alpha_0\bo{d}_0) - \bo{b}\\
            &= \bo{r}_0 + \alpha_0\bo{Ad}_0,
  \end{split}
\end{equation}
and $\bo{r}_1$ is orthogonal to $\bo{d}_0$. Writing the product
$\bo{d}_0\bo{r}_1=0$ as 
\begin{equation}
  \label{eq:14}
  \begin{split}
  \bo{d}_0'\bo{r}_1 &= \bo{d}_0'\bo{r}_0 + \alpha_0\bo{d}_0'\bo{Ad}_0\\
                 &= 0
  \end{split}
\end{equation}
shows that 
\begin{equation}
  \label{eq:15}
  \alpha_0 = \frac{- \bo{d}'_0\bo{r}_0}{\bo{d}_0'\bo{Ad}_0},
\end{equation}
and in general,
\begin{equation}
  \label{eq:16}
    \alpha_i = \frac{- \bo{d}_i'\bo{r}_i}{\bo{d}_i'\bo{Ad}_i}. 
\end{equation}

At this point, line minimization
proceeds in the direction $\bo{d}_1$, and the value of $\bo{x}$ at the minimum
is
\begin{equation}
\label{eq:17}
\bo{x}_2 = \bo{x}_1 + \alpha_1\bo{d}_1.
\end{equation}
The gradient of the function at $\bo{x}_2$ is 
\begin{equation}
\label{eq:18}
\begin{split}
\bo{r}_2 &= \bo{Ax}_2 - \bo{b}\\ 
         &= \bo{r}_1 + \alpha_1\bo{Ad}_1, 
\end{split}
\end{equation}
and $\bo{r}_2$ is orthogonal to $\bo{d}_1$. In the conjugate gradient method,
the direction $\bo{d}_1$ is chosen such that $\bo{r}_2$ will also be orthogonal
to the direction $\bo{d}_0$. Thus the product 
\begin{equation}
  \label{eq:19}
  \begin{split}
    \bo{d}_0'\bo{r}_2 &= \bo{d}_0'\bo{r}_1 + \alpha_1\bo{d}_0'\bo{Ad}_1\\
                    &= \alpha_1\bo{d}_0'\bo{Ad}_1\\
                    &= 0.
  \end{split}
\end{equation}
So, the direction, $\bo{d}_1$ must satisfy the condition
$$
\bo{d}_0'\bo{Ad}_1 = 0
$$
in order for $\bo{r}_2$ to be orthogonal to $\bo{d}_0$. To accomplish this,
following the Grahm-Schmidt procedure, $\bo{d}_1$ is written as
\begin{equation}
  \label{eq:20}
  \bo{d}_1 = \bo{r}_1 - \beta_{10}\bo{d}_0.
\end{equation}
Writing $\bo{d}_0'\bo{Ad}_1=0$ as
\begin{equation}
  \label{eq:21}
  \begin{split}
   \bo{d}_0'\bo{Ad}_1 &= \bo{d}_0'\bo{Ar}_1 - \beta_{10}\bo{d}_0'\bo{Ad}_0\\
                    &= 0         
  \end{split}
\end{equation}
shows that 
\begin{equation}
  \label{eq:22}
  \beta_{10} = \frac{\bo{d}_0'\bo{Ar}_1}{\bo{d}_0'\bo{Ad}_0}.
\end{equation}
In general, 
\begin{equation}
  \label{eq:23}
  \bo{r}_{i+1} = \bo{r}_i + \alpha_i\bo{Ad}_i,
\end{equation}
and by induction, $\bo{r}_{i+1}'\bo{d}_j=0$ provided $\bo{d}_i'\bo{Ad}_j=0$ for
$j<i$. This can be achieved by using Grahm-Schmidt as 
\begin{equation}
  \label{eq:24}
  \bo{d}_i = \bo{r}_i - \sum_{j=0}^{i-1} \beta_{ij}\bo{d}_j,
\end{equation}
where
\begin{equation}
  \label{eq:25}
   \beta_{ij} = \frac{\bo{d}_j'\bo{Ar}_{i}}{\bo{d}_j'\bo{Ad}_j}.
\end{equation}
Note that using the result $\bo{r}_i\bo{d}_j=0$ for $j<i$ in \eqref{eq:24}
implies:
\begin{equation}
  \label{eq:26}
  \bo{r}_i'\bo{r}_j=0\, \text{for}\,  j<i.
\end{equation}
Using (\ref{eq:26}) in (\ref{eq:23}), shows that $\bo{r}_i'\bo{Ad}_j=0$ for
$j<i-1$.  Thus, in (\ref{eq:25}), $\beta_{ij}=0$ for $j<i-1$, and the general
expression for $\bo{d}_i$ simplifies to
\begin{equation}
  \label{eq:27}
  \bo{d}_i = \bo{r}_i - \beta_{i-1}\bo{d}_{i-1},
\end{equation}
where
\begin{equation}
  \label{eq:28}
  \beta_{i-1} = \frac{\bo{d}_{i-1}'\bo{Ar}_i}{\bo{d}_{i-1}'\bo{Ad}_{i-1}}.
\end{equation}
Writing $\bo{r}_i$ as 
\begin{equation}
  \label{eq:29}
  \bo{r}_i = \bo{r}_{i-1} + \alpha_{i-1}\bo{Ad}_{i-1}
\end{equation}
and pre-multiplying by $\bo{r}_i'$ gives
\begin{equation}
  \label{eq:30}
  \bo{r}_i'\bo{r}_i = \alpha_{i-1}\bo{r}_i'\bo{Ad}_{i-1}.
\end{equation}
Pre-multiplying (\ref{eq:29}) by $\bo{d}_{i-1}$ gives
\begin{equation}
  \label{eq:31}
  \bo{d}_{i-1}'\bo{r}_{i-1} = -\alpha_{i-1}\bo{d}_{i-1}'\bo{Ad}_{i-1}.
\end{equation}
Further, from (\ref{eq:27}), we can see that 
\begin{equation}
  \label{eq:32}
  \bo{d}_i'\bo{r}_i = \bo{r}_i'\bo{r}_i.
\end{equation}
Using this is (\ref{eq:31}) gives
\begin{equation}
  \label{eq:33}
  \bo{r}_{i-1}'\bo{r}_{i-1} = -\alpha_{i-1}\bo{d}_{i-1}'\bo{Ad}_{i-1}.
\end{equation}
Using (\ref{eq:30}) and (\ref{eq:33}) in (\ref{eq:28}) gives
\begin{equation}
  \label{eq:34}
  \beta_{i-1} = \frac{-\bo{r}_i'\bo{r}_i}{\bo{r}_{i-1}'\bo{r}_{i-1}}.
\end{equation}
Finally, using (\ref{eq:32}) in (\ref{eq:16}) gives
\begin{equation}
  \label{eq:35}
  \alpha_i = \frac{-\bo{r}_i'\bo{r}_i}{\bo{d}_i'\bo{Ad}_i}.
\end{equation}

#### Preconditioned conjugate gradient method
In the PCCG method, the conjugate gradient method is applied to a transformed
system of equations. The transformation of the system is based on a matrix
$\mathbf{M}$ that is approximately equal to $\mathbf{A}$ and is easy to invert.  A detailed
explanation of PCCG is given in **An Introduction to the Conjugate Gradient
Method Without the Agonizing Pain** by Jonathan Richard Shewchuk.

In PCCG, the solution at iteration $n+1$ is:
\begin{equation}
  \label{eq:36}
  \bo{x}_{n+1} = \bo{x}_n + \alpha_n\bo{d}_n,
\end{equation}
where
\begin{equation}
  \label{eq:37}
  \alpha_n = \frac{-\bo{r}_n'\bo{M}^{-1}\bo{r}_n}
            {\bo{d}_n'\bo{Ad}_n},
\end{equation}
\begin{equation}
  \label{eq:38}
  \bo{r}_n = \bo{Ax}_n - \bo{b},  
\end{equation}
\begin{equation}
  \label{eq:39}
  \bo{d}_n = \bo{M}^{-1}\bo{r}_n - \beta_{n-1}\bo{d}_{n-1}, 
\end{equation}
\begin{equation}
  \label{eq:40}
  \beta_{n-1} = \frac{-\bo{r}_n'\bo{M}^{-1}\bo{r}_n}
                {\bo{r}'_{n-1}\bo{M}^{-1}\bo{r}_{n-1}}.  
\end{equation}
As in the conjugate gradient method, the residual can be computed more
efficiently as
\begin{equation}
  \label{eq:41}
  \bo{r}_n = \bo{r}_{n-1} + \alpha_{n-1}\bo{Ad}_{n-1}.  
\end{equation}
However, it is recommended that \eqref{eq:38} is used to every 50 iterations to
avoid the accumulation of errors.
